In [3]:
# 라이브러리
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series
import plotly.offline as pyo
import plotly.graph_objs as go
import zipfile
from tqdm import tqdm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
import datetime
plt.rc("font", family="Malgun Gothic") # window
plt.rc("axes", unicode_minus=False) # x,y축 (-)부호 표시

#1.데이터 불러오기

In [ ]:
train

In [ ]:
test

In [ ]:
result

#2.전처리

In [ ]:
train.columns = ['ds', 'y']

In [ ]:
train = train.astype({"ds":'str'})
train['year'] = [date.split('-')[0] for date in train['ds']]
train['month'] = [date.split('-')[1] for date in train['ds']]
train['day'] = [date.split('-')[2] for date in train['ds']]
train['day'] = [date.split(' ')[0] for date in train['day']]
train['hour'] = [date.split(' ')[-1] for date in train['ds']]
train['hour'] = [date.split(':')[0] for date in train['hour']]
train = train.drop(['ds'], axis = 1)
train[['year', 'month', 'day', 'hour']] = train[['year', 'month', 'day', 'hour']].astype(int)
train

##1.계절별로 전처리(3-5 : 0, 6-8 : 1, 9-11 : 2, 12-2 : 3)

In [10]:
dic_month2season = {1: 3, 
                    2: 3, 
                    3: 0, 
                    4: 0, 
                    5: 0, 
                    6: 1, 
                    7: 1, 
                    8: 1, 
                    9: 2, 
                    10: 2, 
                    11: 2, 
                    12: 3,
                   }

In [12]:
train["season"] = train["month"].map(dic_month2season)

#3.LinearRegression(결측치 : 0)

##1.Train

In [13]:
train['y'] = train["y"].fillna(0)

In [14]:
X_train = train[["year", "month", "day", "hour", "season"]]
y_train = train["y"]

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
LR = LinearRegression()

In [17]:
LR.fit(X_train, y_train)

LinearRegression()

##2.Test셋

###1.예측할 날짜 추가(마지막날이 없어서)

In [18]:
lst_time = []
current_time = datetime.datetime.strptime(result["datetime"].loc[8424], "%Y-%m-%d %H:%M:%S")
lst_time.append(current_time)

for i in range(335):
    one_hour_later = current_time + datetime.timedelta(hours=1)
    current_time = one_hour_later
    lst_time.append(one_hour_later)

In [19]:
plus_time = pd.DataFrame(lst_time, columns=["datetime"])

In [20]:
test = pd.concat([test, plus_time], axis=0)
test["datetime"] = test["datetime"].astype(str)
test.reset_index(drop=True, inplace=True)

In [21]:
test.columns = ["ds", "y"]

###2.계절별로 전처리(3-5 : 0, 6-8 : 1, 9-11 : 2, 12-2 : 3)

In [22]:
test = test.astype({"ds":'str'})
test['year'] = [date.split('-')[0] for date in test['ds']]
test['month'] = [date.split('-')[1] for date in test['ds']]
test['day'] = [date.split('-')[2] for date in test['ds']]
test['day'] = [date.split(' ')[0] for date in test['day']]
test['hour'] = [date.split(' ')[-1] for date in test['ds']]
test['hour'] = [date.split(':')[0] for date in test['hour']]
test = test.drop(['ds'], axis = 1)
test[['year', 'month', 'day', 'hour']] = test[['year', 'month', 'day', 'hour']].astype(int)

In [23]:
test["season"] = test["month"].map(dic_month2season)

In [24]:
X_test = test[["year", "month", "day", "hour", "season"]]
y_test = test['y']

##3..predict

In [25]:
y_pred = LR.predict(X_test)

In [26]:
y_pred

array([ 239.84183222,  287.38034587,  334.91885953, ..., 1785.58412288,
       1833.12263654, 1880.6611502 ])

In [27]:
df_y_pred = pd.DataFrame(data=y_pred, columns=["y_pred"])

In [28]:
lst_data = []

for i in range(0, len(y_pred)-335):
    data = df_y_pred["y_pred"].loc[i:i+335].values.tolist()
    lst_data.append(data)


In [ ]:
df_submission = pd.DataFrame(data=lst_data)
df_submission

##4.Submission

In [ ]:
file = pd.concat([result["datetime"], df_submission], axis=1)
file.columns = result.columns
file

##5.평가

In [31]:
from sklearn.metrics import mean_absolute_error

In [ ]:
y_pred
df_y_pred

In [33]:
test['y'] = test["y"].fillna(0)

In [34]:
mean_absolute_error(y_test, y_pred)

940.7204578413583

In [35]:
lst_data = []

for i in range(0, len(y_pred)-335):
    data = y_test.loc[i:i+335].values.tolist()
    lst_data.append(data)

In [36]:
submission_test = pd.DataFrame(lst_data, columns=result.columns[1:])

In [37]:
submission_test = pd.concat([result["datetime"], submission_test], axis=1)

In [38]:
evaluation1 = submission_test.dropna()
evaluation2 = file.loc[:8424]

In [ ]:
mission_result=evaluation2[evaluation2.columns[:]]
mission_result

In [ ]:
mission_result.to_csv('LinearRegression.csv',encoding='CP949',index=False)